In [0]:
from pyspark.sql.functions import explode, mean, stddev, col, trim


### Population data from JSON file to dataframe

In [0]:

# Get the most recent JSON file
files = dbutils.fs.ls("/Volumes/thequest/bronze/datausa_io")
latest_file = sorted([f for f in files if f.name.endswith('.json')], key=lambda x: x.name, reverse=True)[0]

# Read as multi-line JSON
df_raw = spark.read.option("multiLine", "true").json(latest_file.path)

# Extract the data array
df_population = (df_raw
  .select(explode(col("data.data")).alias("record"))
  .select("record.*")
)

display(df_population)

### BLS Data as a dataframe

In [0]:
df_bls_current = spark.read.table('thequest.bronze.bls_pr_data_current')

display(df_bls_current.limit(100))

### mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive

In [0]:
# Filter to years 2013-2018 and calculate mean and standard deviation

population_mean_stddev = (df_population
  .filter((col("Year") >= 2013) & (col("Year") <= 2018))
  .select(
    mean("Population").alias("mean_population"),
    stddev("Population").alias("stddev_population")
  )
)

display(population_mean_stddev)

### Best year calc

In [0]:
%sql
WITH yearly_sums AS (
  SELECT 
    TRIM(series_id) as series_id,
    CAST(TRIM(year) as INT) as year,
    SUM(CAST(TRIM(value) as DOUBLE)) as total_value
  FROM thequest.bronze.bls_pr_data_current
  WHERE TRIM(period) LIKE 'Q%'
  GROUP BY TRIM(series_id), CAST(TRIM(year) as INT)
),
max_per_series AS (
  SELECT 
    series_id,
    year,
    total_value,
    MAX(total_value) OVER (PARTITION BY series_id) as max_value
  FROM yearly_sums
)
SELECT 
  series_id,
  year,
  total_value as value
FROM max_per_series
WHERE total_value = max_value
ORDER BY series_id, year




### Population with series data

In [0]:

result = (df_bls_current.alias("b")
  .join(df_population.alias("p"), 
        col("b.year") == col("p.year"), 
        "inner")
  .filter((trim(col("b.series_id")) == "PRS30006032") & 
          (trim(col("b.period")) == "Q01"))
  .select(
    col("b.series_id"),
    col("b.year"),
    col("b.period"),
    col("b.value"),
    col("p.population")
  )
)

display(result)